<a href="https://colab.research.google.com/github/mnJ00/Data/blob/main/%EC%95%84%ED%8C%8C%ED%8A%B8_%EC%8B%A4%EA%B1%B0%EB%9E%98%EA%B0%80_%EC%98%88%EC%B8%A1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 아파트 실거래가 예측

## EDA

In [ ]:
import pandas as pd

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
submission = pd.read_csv('sample_submission.csv')

In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 15 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   user_id                            10000 non-null  object 
 1   subscription_duration              10000 non-null  int64  
 2   recent_login_time                  10000 non-null  int64  
 3   average_login_time                 10000 non-null  float64
 4   average_time_per_learning_session  10000 non-null  float64
 5   monthly_active_learning_days       10000 non-null  int64  
 6   total_completed_courses            10000 non-null  int64  
 7   recent_learning_achievement        10000 non-null  float64
 8   abandoned_learning_sessions        10000 non-null  int64  
 9   community_engagement_level         10000 non-null  int64  
 10  preferred_difficulty_level         10000 non-null  object 
 11  subscription_type                  10000 non-null  obje

In [ ]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   user_id                            10000 non-null  object 
 1   subscription_duration              10000 non-null  int64  
 2   recent_login_time                  10000 non-null  int64  
 3   average_login_time                 10000 non-null  float64
 4   average_time_per_learning_session  10000 non-null  float64
 5   monthly_active_learning_days       10000 non-null  int64  
 6   total_completed_courses            10000 non-null  int64  
 7   recent_learning_achievement        10000 non-null  float64
 8   abandoned_learning_sessions        10000 non-null  int64  
 9   community_engagement_level         10000 non-null  int64  
 10  preferred_difficulty_level         10000 non-null  object 
 11  subscription_type                  10000 non-null  obje

In [ ]:
null_data = train[train['exclusive_use_area'].isnull()]
null_data

KeyError: 'exclusive_use_area'

In [ ]:
## 컬럼간 관계 탐색: 도수분포(히스토그램)
import matplotlib.pyplot as plt

plt.figure(figsize = (6,6))

x = train[train['apartment_id'] == 2646]['exclusive_use_area']
plt.hist(x, label='histogram')

plt.xlabel('exclusive_use_area')
plt.ylabel('number of apartment')

plt.legend()
plt.show()

In [ ]:
## 결측값 대치 apartment_id의 median

median_value = train.groupby('apartment_id')['exclusive_use_area'].median()
median_value

In [ ]:
## 결측값(Null) 대치(2): merge()

null_filled = pd.merge(median_value, null_data, on='apartment_id')
null_filled

In [ ]:
## 결측값(Null) 대치(3): 형태 통일하기

null_filled = null_filled.rename(columns={'exclusive_use_area_x':'exclusive_use_area'})
null_filled = null_filled.drop(columns=['exclusive_use_area_y'])

In [ ]:
## 결측값 대치 완료 데이터 만들기
train_cleaned = train.dropna()
train_filled = pd.concat([train_cleaned, null_filled])

In [ ]:
## 데이터 전처리: 칼럼 선정

train_x = train_filled.drop(columns=['transaction_id', 'apartment_id', 'addr_kr', 'transaction_real_price'])
train_y = train_filled['transaction_real_price']

## 데이터 전처리: replace
train_x = train_x.replace({'transaction_date': '1~10'} , 1)
train_x = train_x.replace({'transaction_date': '11~20'} , 2)
train_x = train_x.replace({'transaction_date': '21~31'} , 3)
train_x = train_x.replace({'transaction_date': '21~29'} , 3)
train_x = train_x.replace({'transaction_date': '21~30'} , 3)
train_x = train_x.replace({'transaction_date': '21~28'} , 3)

In [ ]:
## 모델 학습 전 데이터 전처리(3): Label Encoding

from sklearn.preprocessing import LabelEncoder

le_city = LabelEncoder()
le_city = le_city.fit(train_x['city'])
train_x['city'] = le_city.transform(train_x['city'])

le_dong = LabelEncoder()
le_dong = le_dong.fit(train_x['dong'])
train_x['dong'] = le_dong.transform(train_x['dong'])

le_jibun = LabelEncoder()
le_jibun = le_jibun.fit(train_x['jibun'])
train_x['jibun'] = le_jibun.transform(train_x['jibun'])

le_apt = LabelEncoder()
le_apt = le_apt.fit(train_x['apt'])
train_x['apt'] = le_apt.transform(train_x['apt'])

## 선형회귀 모델 적용

In [ ]:
## 선형회귀 모델 훈련

from sklearn.linear_model import LinearRegression

model = LinearRegression()
model.fit(train_x, train_y)

In [ ]:
## 예측 전 test 데이터 전처리
test = pd.read_csv('test.csv')
test.info()

test_x = test.drop(columns=['transaction_id', 'addr_kr', 'apartment_id'])

test_x = test_x.replace({'transaction_date': '1~10'} , 1)
test_x = test_x.replace({'transaction_date': '11~20'} , 2)
test_x = test_x.replace({'transaction_date': '21~31'} , 3)
test_x = test_x.replace({'transaction_date': '21~29'} , 3)
test_x = test_x.replace({'transaction_date': '21~30'} , 3)
test_x = test_x.replace({'transaction_date': '21~28'} , 3)

import numpy as np

for city in np.unique(test_x['city']):
    if city not in le_city.classes_:
        le_city.classes_ = np.append(le_city.classes_, city)
test_x['city'] = le_city.transform(test_x['city'])

for dong in np.unique(test_x['dong']):
    if dong not in le_dong.classes_:
        le_dong.classes_ = np.append(le_dong.classes_, dong)
test_x['dong'] = le_dong.transform(test_x['dong'])

for jibun in np.unique(test_x['jibun']):
    if jibun not in le_jibun.classes_:
        le_jibun.classes_ = np.append(le_jibun.classes_, jibun)
test_x['jibun'] = le_jibun.transform(test_x['jibun'])

for apt in np.unique(test_x['apt']):
    if apt not in le_apt.classes_:
        le_apt.classes_ = np.append(le_apt.classes_, apt)
test_x['apt'] = le_apt.transform(test_x['apt'])

In [ ]:
## 모델을 이용한 예측: test 데이터에 대해서 예측
prediction = model.predict(test_x)

submission = pd.read_csv('submission.csv')

submission['transaction_real_price'] = prediction

prediction

In [ ]:
null_data = train[train['exclusive_use_area'].isnull()]
null_data.count()

In [ ]:
## 검증데이터 만들기
from sklearn.model_selection import train_test_split

train_split, validation = train_test_split(train, train_size=0.7)

train_split = train_split.dropna()
validation = validation.dropna()

In [ ]:
## 데이터 전처리, 활용 컬럼 선정
train_x = train_split[['exclusive_use_area', 'year_of_completion', 'transaction_year_month', 'floor', 'apartment_id']]
train_y = train_split['transaction_real_price']

valid_x = validation[['exclusive_use_area', 'year_of_completion', 'transaction_year_month', 'floor', 'apartment_id']]
valid_y = validation['transaction_real_price']

##StandardScaler

In [ ]:
## 데이터 전처리(표준화)
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

scaler.fit(train_x)
train_x_scaled = scaler.transform(train_x)
valid_x_scaled = scaler.transform(valid_x)

In [ ]:
## 데이터 전처리 Principal Component Analysis, (1차원)

from sklearn.decomposition import PCA

pca = PCA(n_components=1)

pca.fit(train_x_scaled)
train_x_pca = pca.transform(train_x_scaled)
valid_x_pca = pca.transform(valid_x_scaled)

In [ ]:
## LinearRegression 모델 평가

from sklearn.linear_model import LinearRegression

model_lr = LinearRegression()
model_lr.fit(train_x_pca, train_y)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize = (12,6))

# 학습 데이터 시각화
plt.scatter(train_x_pca, train_y, label='train data') #1
plt.xlabel('train_x')
plt.ylabel('train_y')

# 모델의 학습 현황 시각화
x_sorted = train_x_pca.tolist() #2
x_sorted.sort() #3
pred_lr = model_lr.predict(x_sorted) #4
plt.plot(x_sorted, pred_lr, color='red', label='linear regression') #5

plt.legend()
plt.show()

In [ ]:
plt.figure(figsize = (12,6))

plt.scatter(valid_x_pca, valid_y, label='validation data')
plt.xlabel('valid_x')
plt.ylabel('valid_y')

x_sorted = valid_x_pca.tolist()
x_sorted.sort()
pred_lr = model_lr.predict(x_sorted)
plt.plot(x_sorted, pred_lr, color='red', label='linear regression')

plt.legend()
plt.show()

 ## Decision Tree 모델 적용


In [ ]:
## . Decision Tree 모델 평가

from sklearn.tree import DecisionTreeRegressor

model_dt = DecisionTreeRegressor()
model_dt.fit(train_x_pca, train_y)

In [ ]:
## train 데이터
plt.figure(figsize = (12,6))

plt.scatter(train_x_pca, train_y, label='train data')
plt.xlabel('train_x')
plt.ylabel('train_y')

x_sorted = train_x_pca.tolist()
x_sorted.sort()
pred_dt = model_dt.predict(x_sorted)
plt.plot(x_sorted, pred_dt, color='red', label='decision tree')

plt.legend()
plt.show()

In [ ]:
## 검증데이터
plt.figure(figsize = (12,6))

plt.scatter(valid_x_pca, valid_y, label='validation data')
plt.xlabel('valid_x')
plt.ylabel('valid_y')

x_sorted = valid_x_pca.tolist()
x_sorted.sort()
pred_dt = model_dt.predict(x_sorted)
plt.plot(x_sorted, pred_dt, color='red', label='decision tree')

plt.legend()
plt.show()

## 랜덤포레스트 적용 전 데이터 전처리

In [ ]:
train.isnull().sum()


In [ ]:
train = train.dropna(subset=['transaction_year_month'])
train['transaction_year_month'].fillna('0', inplace=True)

In [ ]:
## 랜덤 포레스트 준비1, 데이터 전처리 -파생변수 생성
import pandas as pd

train['transaction_year'] = train['transaction_year_month'].astype(str).str[:4].astype(int)
train['transaction_month'] = train['transaction_year_month'].astype(str).str[4:].astype(int)

In [ ]:
## 데이터전처리-결측값 대치
null_data = train[train['exclusive_use_area'].isnull()]
median_value = train.groupby('apartment_id')['exclusive_use_area'].median()

null_filled = pd.merge(median_value, null_data, on='apartment_id')
null_filled = null_filled.rename(columns={'exclusive_use_area_x':'exclusive_use_area'})
null_filled = null_filled.drop(columns=['exclusive_use_area_y'])

train_cleaned = train.dropna()
train_filled = pd.concat([train_cleaned, null_filled])

In [ ]:
## 모델 핛습 전 칼럼 선정
train_x = train_filled.drop(columns=['transaction_id', 'apartment_id', 'addr_kr', 'transaction_year_month', 'transaction_real_price'])
train_y = train_filled['transaction_real_price']

In [ ]:
## 객체형 데이터 수치화
train_x = train_x.replace({'transaction_date': '1~10'} , 1)
train_x = train_x.replace({'transaction_date': '11~20'} , 2)
train_x = train_x.replace({'transaction_date': '21~31'} , 3)
train_x = train_x.replace({'transaction_date': '21~29'} , 3)
train_x = train_x.replace({'transaction_date': '21~30'} , 3)
train_x = train_x.replace({'transaction_date': '21~28'} , 3)

In [ ]:
from sklearn.preprocessing import LabelEncoder

le_city = LabelEncoder()
le_city = le_city.fit(train_x['city'])
train_x['city'] = le_city.transform(train_x['city'])

le_dong = LabelEncoder()
le_dong = le_dong.fit(train_x['dong'])
train_x['dong'] = le_dong.transform(train_x['dong'])

le_jibun = LabelEncoder()
le_jibun = le_jibun.fit(train_x['jibun'])
train_x['jibun'] = le_jibun.transform(train_x['jibun'])

le_apt = LabelEncoder()
le_apt = le_apt.fit(train_x['apt'])
train_x['apt'] = le_apt.transform(train_x['apt'])

## 랜덤 포레스트 모델 검증

In [ ]:
## 랜덤포레스트 모델

from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(random_state=777)

In [ ]:
## 하이퍼 파라미터 튜닝: Gird Search Cross Validation

from sklearn.model_selection import GridSearchCV

param_grid = [{
    'n_estimators': [9, 13],
    'max_depth': [29, 33]
    }
]

grid_search = GridSearchCV(
    model,
    param_grid,
    cv=2,
    scoring='neg_root_mean_squared_error'
)

grid_search.fit(train_x, train_y)
print('Done.')

In [ ]:
## 하이퍼 파라미터 튜닝 결과 확인
result = pd.DataFrame(grid_search.cv_results_)
result[['params', 'split0_test_score', 'split1_test_score', 'mean_test_score', 'rank_test_score']]

In [ ]:
## 최적 모델 학습

best_model = RandomForestRegressor(n_estimators=9, max_depth=33, random_state=777)
best_model.fit(train_x, train_y)


In [ ]:
## 예측 전 test데이터 전처리

test = pd.read_csv('test.csv')
test['transaction_year'] = test['transaction_year_month'].astype(str).str[:4].astype(int)
test['transaction_month'] = test['transaction_year_month'].astype(str).str[4:].astype(int)

test_x = test.drop(columns=['transaction_id', 'addr_kr', 'apartment_id', 'transaction_year_month'])

test_x = test_x.replace({'transaction_date': '1~10'} , 1)
test_x = test_x.replace({'transaction_date': '11~20'} , 2)
test_x = test_x.replace({'transaction_date': '21~31'} , 3)
test_x = test_x.replace({'transaction_date': '21~29'} , 3)
test_x = test_x.replace({'transaction_date': '21~30'} , 3)
test_x = test_x.replace({'transaction_date': '21~28'} , 3)

import numpy as np

for city in np.unique(test_x['city']):
    if city not in le_city.classes_:
        le_city.classes_ = np.append(le_city.classes_, city)
test_x['city'] = le_city.transform(test_x['city'])

for dong in np.unique(test_x['dong']):
    if dong not in le_dong.classes_:
        le_dong.classes_ = np.append(le_dong.classes_, dong)
test_x['dong'] = le_dong.transform(test_x['dong'])

for jibun in np.unique(test_x['jibun']):
    if jibun not in le_jibun.classes_:
        le_jibun.classes_ = np.append(le_jibun.classes_, jibun)
test_x['jibun'] = le_jibun.transform(test_x['jibun'])

for apt in np.unique(test_x['apt']):
    if apt not in le_apt.classes_:
        le_apt.classes_ = np.append(le_apt.classes_, apt)
test_x['apt'] = le_apt.transform(test_x['apt'])


In [ ]:
## 랜덤포레스트 모델 이용 예측

prediction = best_model.predict(test_x)

submission = pd.read_csv('submission.csv')
submission['transaction_real_price'] = prediction

In [ ]:
submission